# Домашнее задание 1

Даны цены на следующие облигации:

* Бескупонная облигация сроком на полгода: 98708.4135
* Облигация на год, с купонным платежом 1.5%, раз в полгода: 99980.8479
* Облигация на 2 года, с купонным платежом 1.5%, раз в полгода: 98055.8851
* Облигация на 3 года, с купонным платежом 1%, раз в полгода: 92706.3023

Все облигации номиналом 100000.

Расчитать процентные ставки для кривой доходности для следующих интервалов времени: полгода, год, 2 года, 3 года.
Использовать формулу непрерывного начисления процентов.

Ответ показать в виде таблицы.

In [1]:
import numpy as np
import pandas as pd
import scipy.interpolate
import scipy.optimize

In [3]:
N = 100000
prices = np.array([98708.4135, 99980.8479, 98055.8851, 92706.3023])
rates_bond = np.array([0, 1.5, 1.5, 1])/100*N

In [4]:
def DF(r_t,t):
    if callable(r_t):
        r = r_t(t)
    else:
        r = r_t
    return np.exp(-r*t)

def r_DF(DF, t):
    return - np.log(DF)/t

In [5]:
DF_06 = prices[0] / N
rates06 = r_DF(DF_06, 1/2)

In [6]:
DF_12 = (prices[1] - rates_bond[1]*DF_06)/(N+rates_bond[1])
rates12 = r_DF(DF_12, 1)

In [8]:
def diff_B_24_for_r_24(r_24):
    DF_24 = DF(r_24, 2)
    r_18 = (r_24+rates12)/2
    DF_18 = DF(r_18, 1.5)
    c = rates_bond[2]
    #print((c*DF_06 + c*DF_12 + c*DF_18 + (c+N)*DF_24))
    return prices[2] - (c*DF_06 + c*DF_12 + c*DF_18 + (c+N)*DF_24)

sol = scipy.optimize.root(diff_B_24_for_r_24, rates12)

rates24 = sol.x[0]


In [9]:
rates18 = (rates12+rates24)/2
DF_18 = DF(rates18, 1.5)
DF_24 = DF(rates24, 2)

In [10]:
def diff_B_36_for_r_36(r_36):
    DF_36 = DF(r_36, 3)
    r_30 = (r_36 + rates24)/2
    DF_30 = DF(r_30, 2.5)
    c = rates_bond[3]
    return prices[3] - (c*DF_06 + c*DF_12 + c*DF_18 + c*DF_24 + c*DF_30 + (c+N)*DF_36)

sol = scipy.optimize.root(diff_B_36_for_r_36, rates24)

rates36 = sol.x[0]

In [15]:
rates = np.array([rates06, rates12, rates24, rates36])
rates = np.around(rates, decimals=4)
times = np.array([0.5, 1, 2, 3])

In [16]:
pd.DataFrame(data = (times, rates), index=("time (y)", "rate (%)"))

,0,1,2,3
time (y),0.500,1.00,2.00,3.000
rate (%),0.026,0.03,0.04,0.046
